In [1]:
!pip install pydub

You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [1]:
import glob
from pydub import AudioSegment
import pandas as pd
import json
import numpy as np
import os
import re

In [2]:
DATA_DIR = '/Users/terlanismayilsoy/Education/Ders/SPRING 2021/Senior Design Project II/Local Samples/Training Files/'
AUDIO_OUTPUT_DIR = '/Users/terlanismayilsoy/Education/Ders/SPRING 2021/Senior Design Project II/Local Samples/Audio Output/'

BOOKS = [
         'Bulanlıq Dünya',
         'Səfillər'
]

In [ ]:
# FOR TESTING FILE MATCHES

In [ ]:
counter = 0

for book_name in BOOKS:
    book_path = DATA_DIR + book_name + '/'
    
    syncmap_paths = sorted(glob.glob(book_path + '**/*.json', recursive=True))
    audio_paths = sorted(glob.glob(book_path + '**/*.mp3', recursive=True))
    
#     pattern = re.compile('\d*')
    pattern = re.compile('book.*')
#     print(syncmap_paths, audio_paths)
    for syncmap_path, audio_path in zip(syncmap_paths, audio_paths):
#         print(os.path.basename(syncmap_path), os.path.basename(audio_path))
#         syncMatch, audioMatch = pattern.match(os.path.basename(syncmap_path)), pattern.match(os.path.basename(audio_path))
        syncMatch, audioMatch = pattern.search(syncmap_path), pattern.search(audio_path)
        
        print(syncMatch.group(0), '->' , audioMatch.group(0))
        
#         if syncMatch.group(0) != audioMatch.group(0):
#             print(syncmap_path, audio_path)
            
#         counter += 1
        
# print(counter)
        

In [3]:
# NORMALIZATION
from num2words import num2words
import pandas as pd
import numpy as np
import re

def normalize_num(num: str, is_ordinal: bool):
	"""Converts the given number to words"""
	
	if num == "0":
		return 'sıfır'
	
	turkish_str = num2words(int(num), lang='tr', ordinal = is_ordinal)

	aze_str = translate_to_aze(turkish_str)
	
	return aze_str

def translate_to_aze(turkish_str: str):
	"""Translates the given turkish string to azerbaijani"""
	
	aze_str = turkish_str
	
	aze_str = aze_str.replace('dört', 'dörd')
	aze_str = aze_str.replace('yedi', 'yeddi')
	aze_str = aze_str.replace('sekiz', 'səkkiz')
	aze_str = aze_str.replace('dokuz', 'doqquz')
	aze_str = aze_str.replace('yirmi', 'iyirmi')
	aze_str = aze_str.replace('kırk', 'qırx')
	aze_str = aze_str.replace('elli', 'əlli')
	aze_str = aze_str.replace('seksen', 'səksən')
	aze_str = aze_str.replace('doksan', 'doxsan')
	aze_str = aze_str.replace('bin', 'min')
	aze_str = aze_str.replace('milyar', 'milyard')
	aze_str = aze_str.replace('katrilyon', 'kvadrilyon')
	aze_str = aze_str.replace('kentilyon', 'kvintilyon')
	
	return aze_str

In [4]:
FILE_COUNTER = 1
df = pd.DataFrame(columns=['filename', 'text', 'normalized_text'])

for book_name in BOOKS:
    book_path = DATA_DIR + book_name + '/'
    
    syncmap_paths = sorted(glob.glob(book_path + '**/*.json', recursive=True))
    audio_paths = sorted(glob.glob(book_path + '**/*.mp3', recursive=True));
    
#     print(*zip(syncmap_paths, audio_paths))
    
    for syncmap_path, audio_path in zip(syncmap_paths, audio_paths):
        audio_segment = AudioSegment.from_mp3(audio_path)
        audio_segment = audio_segment.set_channels(1)
        audio_segment = audio_segment.set_frame_rate(22050)
        
        with open(syncmap_path) as file:
            syncmap = json.loads(file.read())

        sentences = []

        for fragment in syncmap['fragments']:
            sentences.append({'audio': audio_segment[float(fragment['begin']) * 1000 : float(fragment['end']) * 1000], 'text': fragment['lines'][0]})

        for idx, sentence in enumerate(sentences):
#             if FILE_COUNTER % 1000 == 0:
#                 print('Current segment: {}'.format(FILE_COUNTER))

            text = sentence['text'].lower()
            duration = sentence['audio'].duration_seconds
            
            if duration > 10:
                continue
        
            if type(text) is not str: # deleting empty string
                continue
            
            matches = re.findall('(\d+-([a-zöəüğçşı]{3}|[a-zöəüğçşı]{2})|\d+)', text)
            
            sentence_new = text
            
            if matches:
                for match in matches:
                    num, group = match
                    num = re.search('\d+', num)[0]

                    if group in ['cı', 'ci', 'cu', 'cü']:
                        sentence_new = text.replace(match[0], normalize_num(num, True)[:-1])
                    else:
                        sentence_new = text.replace(match[0], normalize_num(num, False)[:-1] + group)
            
            sentence['audio'].export(AUDIO_OUTPUT_DIR + 'sample-' + str(FILE_COUNTER + idx) + '.wav', format='wav')
            temp_df = pd.DataFrame([{'filename': 'sample-' + str(FILE_COUNTER + idx), 'text': text, 'normalized_text' : sentence_new}], columns=['filename','text', 'normalized_text'])
            df = df.append(temp_df, ignore_index=True)

        FILE_COUNTER += len(sentences)



In [5]:
df

,filename,text,normalized_text
0,sample-1,"nadir nüsxə olan, başqa kitabxanalarda tapılma...","nadir nüsxə olan, başqa kitabxanalarda tapılma..."
1,sample-2,ona buranı salıx vermişdilər.,ona buranı salıx vermişdilər.
2,sample-3,əfsun i̇qtisad i̇nstitutunda təhsil alırdı.,əfsun i̇qtisad i̇nstitutunda təhsil alırdı.
3,sample-4,“maliyyə-kredit” fakültəsinin üçüncü kurs tələ...,“maliyyə-kredit” fakültəsinin üçüncü kurs tələ...
4,sample-5,əla qiymətlərlə oxuyurdu.,əla qiymətlərlə oxuyurdu.
...,...,...,...
27286,sample-27797,i̇ctimai qayda-qanun ciddi bir şəkildə pozulsa...,i̇ctimai qayda-qanun ciddi bir şəkildə pozulsa...
27287,sample-27798,"monastıra gəlincə, onun foşlevana bəslədiyi mi...","monastıra gəlincə, onun foşlevana bəslədiyi mi..."
27288,sample-27799,foşlevan monastırın ən yaxşı xidmətçilərindən ...,foşlevan monastırın ən yaxşı xidmətçilərindən ...
27289,sample-27802,"foşlevanın şöhrəti bununla da qurtarmadı, lap ...","foşlevanın şöhrəti bununla da qurtarmadı, lap ..."


In [6]:
np.savetxt('/Users/terlanismayilsoy/Education/Ders/SPRING 2021/Senior Design Project II/Local Samples/metadata_3_col.csv', df, delimiter='|', fmt='%s')

In [17]:
np.savetxt(AUDIO_OUTPUT_DIR + 'metadata.csv', df, delimiter='|', fmt='%s')

In [23]:
from google.colab import files
df.to_csv('metadata.csv') 
files.download('metadata.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
my_drive = GoogleDrive(gauth)

In [ ]:
for a_file in my_drive.ListFile({'q': "title contains 'sample'"}).GetList():
    print(a_file)
    a_file.Delete()